<h1 style=color:blue align="center"> Predict the Severity of Adverse Drug Reactions </h1>

------------

<h2>1. Introduction</h2>
<p>The objective of this work is to develop machine learning (ML) methods that can accurately predict adverse drug reactions (ADRs) using databases like SIDER and OFFSIDES.</p> 
<h2>2. Methods</h2>
<h3>2.1 Methods</h3>
<p>One of the most important factors when using ML methods are the datasets used to train, validate and test the model. In this work, 3 different ones will be used at different stages, shown in table 1.</p> 

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-0lax"><span style="font-weight:bold">Dataset</span></th>
    <th class="tg-0lax"><span style="font-weight:bold">Description</span></th>
  </tr>
  <tr>
    <td class="tg-0lax">SIDER 4</td>
    <td class="tg-0lax">1427 Approved drugs with ADRs text-mined from drug package inserts grouped into 27 system organ classes following <br>MedDRA classification</td>
  </tr>
  <tr>
    <td class="tg-0lax">OFFSIDES</td>
    <td class="tg-0lax">Database of off-label side effects</td>
  </tr>
</table>

When pre-processing OFFSIDES, ADRs were grouped by system organ classes following MedDRA classification and SMILES strings were obtained from PubChem using the REST API and the STITCH IDs of the compounds.

<h3>2.2 Features</h3>
<p>Features are the set of attributes associated with the example that try to represent the dataset.</p> 
<p>SMILES strings are commonly used to represent molecules, as is used in SIDER, and they will be the basis of this work. But, even though they are a unique representation of molecules, they are not enough to use as a feature in ML. Because of this, they will be used as a way to generate other features like fingerprints and molecular descriptors using tools like RDKit in Python. The general workflow for the datasets when in SIDER format is displayed in the following figure.</p> 



<img src="image/sider.png" alt="Drawing" style="width: 600px;"/>

The SIDER dataset consists of a first column with the molecules' SMILES representation and twenty-seven other columns with the different SOCs. Three of these SOCs were not used since they had no real connection with the molecule and, as such, the development of ML models to predict these labels was not useful; these were 'Product Issues', 'Investigations', and 'Social circumstances'. 

With the SMILES representation, it was possible to create multiple different features using RDKit, mainly fingerprints and other descriptors, for example, molecular weight, number of radical electrons, and number of valence electrons. We used these to add relevant information that complements the fingerprint. 

In total, 27 descriptors were calculated for each molecule; not every descriptor was useful and, as such, some selection was required. But, since we had 24 different classification tasks, each with an independent model, and different descriptors had a different importance for each of them, this selection was done independently for each task, which resulted in 24 different DataFrames consisting in the fingerprint representation plus the 3 (after testing different values) descriptors selected for each task. An example is shown in following table:
<img src="image/withdesc.png" alt="Drawing" style="width: 400px;"/>

This selection was done using the SelectKBest function from scikit-learn with ANOVA as the statistical test.
When transforming OFFSIDES and after getting the SMILES from the STITCH IDs, the process is the same as described before.

<h3>2.3 Multinomial classification methods</h3>

To predict drug-drug interaction severity values

<p>Supervised learning is the most common ML scenario in chemoinformatics, and can be subdivided into classification and regression problems. In this type of learning, the training data has the outcome variable to guide the learning process. The objective of this type of learning is to predict the value of an outcome or to classify it. The tested models were:</p> 

<h4>2.3.1 Support Vector Machine (SVM)</h4>
<p>One of the most popular ML methods. It maps the data into a high-dimensional space, using a non-linear kernel function, in order to optimally separate the classes. This separation is done by maximizing the margin between the closest points of the classes, support vectors, to the decision boundary, a hyperplane.</p>

<h4>2.3.2 Naive Bayes </h4>

<h4>2.3.3 Logistic Regression </h4>

<h4>2.3.4 Random Forest (RF)</h4>
<p>Tries to give a classification based on an ensemble of decision trees built based on the training data. It is an ensemble of tree predictors where each tree is independently constructed by using bootstrap samples of the training data and random feature selection.</p>
<p>After the RF is built, a prediction is made by a majority vote or averaging the predictions of all the trees.</p>

<h4> Gradient Boosted Trees (GBT)</h4>
<p>Similar to RF, as it is also an ensemble prediction method but the trees are not independent. This comes from the fact that, in GBT, at each iteration, the respective tree is constructed by fitting a simple function to current residuals.</p>
<p>The models tested and optimized were SVC (classification implementation of SVM) and Random Forest using scikit-learn, and Gradient Boosted Trees with XGBoost.</p>

<h4> Model Development</h4>
<p>As it is possible to see mext, the percentage of positives is very different from label to label. Because of this, the workflow for each model was base evaluation of the base model trained on the original dataset using cross-validation, followed by cross-validation with oversampling of the minority class, followed by hyperparameter optimization using random and grid search, followed by a final validation with the optimized parameters and oversampling. This process is shown next:</p>

----------------

The true labels for these interactions are one of 5 classes from
Lexi-Interact: Avoid combination, Consider Therapy Modification, Monitor Therapy, No Action Needed, and No Known
Interaction

## ML model development and validation:
<img src="image/model2.png" alt="Drawing" style="width: 600px;"/>
After replicating this process for the 3 models, the best one for each label was selected and tested with the test dataset.

<h4>2.3.5 Cross-Validation (cv)</h4>
<p>This process was done using stratified k-fold so that each set contains approximately the same percentage of a sample of each target class as the complete set.</p>

<h4>2.3.4 Class Balancing</h4>
<p>As is seen in figure previous figure, one of the steps when developing the ML models was balancing the dataset. This can be necessary when the classification categories are not approximately equally represented. </p>

Explores **downsampling** from the **majority classes** or **upsampling** the **minority classes**, such that the **training set is balanced and the test set is untouched** ( Bone, et al., 2015 ). This upsampling approach **duplicates** full training examples rather than generating simulated data points. By doing so, it **removes a bias towards the majority classes**, while making no assumptions about the underlying data.

<p>Class imbalance can, usually, be dealt with by re-sample the dataset, either by over-sampling the minority class and/or under-sampling the majority class. In this work, over-sampling was used, specifically an extension of Synthetic Minority Over-sampling TEchnique (SMOTE) with the imbalanced-learn package, SMOTE-NC. </p>

<p>With SMOTE, the minority class is over-sampled by introducing synthetic examples along the line segments joining k minority class nearest neighbours. SMOTE-NC adapts this strategy by doing something specifically for the categorical features. When generating a new sample, it picks the most frequent category of the nearest neighbours present for these features.</p>

<p>There are some dangers when combining class balancing with cross-validation. In order to keep the validation process valid, balancing should not be done before separating train and validation for each fold. That is, we start the cross-validation process, dividing the training set in training and validation, and only then do we over-sample the minority class (when employing over-sampling). We do this for every iteration of the process. This process will minimize possible overfitting and a change in the test distribution that would result in misleading results. </p>

<h3>2.4 Metrics</h3>
<p>Performance measures for classification are typically based on the confusion matrix.</p>
<img src="image/confusionmatrix.png" alt="Drawing" style="width: 200px;"/>

In this matrix, TN are the true negatives, TP are the true positives, FP are the false positives, and FN are the false negatives.

Using this format it is possible to calculate other metrics in order to evaluate the quality of a model's predictions. In order to have a better idea of how well a model works, different metrics should be used. In this work, it was used: Recall, Precision, Average Precision, Area Under the Receiver Operating Characteristic (AUROC), and different variations of F1 score. 

**Precision** is the ability of the classifier not to label as positive a sample that is negative and is defined by:

$$Precision = \frac{\text{TP}}{\text{TP + FP} }$$

**Recall** is the ability of the classifier to label as positive a sample that is positive and is defined by:


$$Recall = \frac{\text{TP}}{\text{TP + FN} }$$


**Average Precision** summarizes a precision-recall curve as the weighted mean of precisions archived at each threshold $(P_{n})$. The increase in recall $(R_{n})$ from the previous threshold $(R_{n-1})$ is used as the weight. It is defined by:


$$AP = \sum_{n} \left ( R_{n} - R_{n-1} \right ) P_{n}$$


**AUROC** is the area under the receiving operating characteristic (ROC) curve. This curve is created by plotting the fraction of TP out of the actual positives against the fraction of FP out of the actual negatives, at different thresholds.
\\

**F1 Score** is a weighted average of the precision and recall and is defined by:


$$F1 = \frac{2 \cdot precision\cdot recall}{precision + recall}$$


In this work, three types of F1-score were used. F1 binary, also represented as F1, is the F1 Score with respect only to the positive label. F1 Macro Score is the unweighted mean between both positive and negative labels. F1 Micro Score uses global TP, FN and FP and is equivalent to the accuracy metric in a binary classification task.

During this work, Average Precision, Recall and the different F1 Scores will be the main metrics used to evaluate and develop the model since they deal with imbalanced datasets better than AUROC.


<h2>3. Results and Discussion</h2>

In this work 24 models were studied, one for each SOC. Before any testing, the dataset was split in train and test and all validation and optimization tasks were done using the first, in order to prevent any type of test overfitting.

Something to have in mind when evaluating these results is the imbalance of the test dataset which is a consequence of the same imbalance of the original dataset. This greatly affects the metrics, mainly the precision and all metrics that derive from it since having a big majority of positive tests will always result in high precision scores.


<h3>3.1 Feature Generation and Selection</h3>

The first step was to choose a fingerprint type and its length. The tested possibilities were ECFP-4, MACCS key, Atom Pairs and Topological Torsion. For each of these types, different lengths between 100 and 2048 were tested and the different metrics were calculated in order to pick the best combination. In order to simplify this process, the different combinations were tested using 10-fold cross-validation with SVC and only to the label 'Hepatobiliary disorders'.
The results are displayed next:
